<a href="https://colab.research.google.com/github/carolyna-s/CS123A_Project/blob/main/projectmethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Edited & Modified from course material

# AI/ML methods notebook



In [ ]:
!pip install scikit-learn --no-cache
!pip install scanpy --no-cache
!pip install gseapy --no-cache
!pip install pybiomart==0.1 --no-cache
!pip install sklearn_som  --no-cache
!pip install pandas --no-cache
!pip install numpy --no-cache
!pip install matplotlib --no-cache
!pip install sklearn-som --no-cache
!pip install pyDeseq2 --no-cache
!pip install Ensembl_converter --no-cache
!pip install shap

# import Python modules

This notebook imports a number of Python modules for use in several notebooks.

In [ ]:
import requests
import json
import pandas as pd
from urllib.request import urlretrieve
import numpy as np
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import scanpy as sc
import gseapy as gp
from gseapy.plot import gseaplot
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from gseapy import Msigdb
from pybiomart import Server
#import mygene
import seaborn as sns
from sklearn.decomposition import PCA, FastICA
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import TweedieRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from math import log
import statsmodels.api as sm
import pylab
import operator
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from itertools import islice
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import argparse
import scipy.stats as stats
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')
from Ensembl_converter import EnsemblConverter
import shap
from sklearn.inspection import permutation_importance
from google.colab import data_table
#from vega_datasets import data

In [ ]:
from itertools import zip_longest

# define misc helper methods

In [ ]:
def set_maxdisplay(n=None):
  pd.set_option('display.max_rows', n)
  from notebook.services.config import ConfigManager
  cm = ConfigManager().update('notebook', {'limit_output': n})

# Define data ingestion methods

In [ ]:
def read_meta_data(dataset):
  # so to read data from OSD-XXX, we would add m.read_meta_data('XXX')
  # dataset=255
  url = 'https://osdr.nasa.gov/geode-py/ws/studies/OSD-' + str(dataset) + '/download?source=datamanager&file=OSD-' + dataset + '_metadata_OSD-' + dataset + '-ISA.zip'
  filename = dataset + '-meta.zip'
  urlretrieve(url, filename)
  !unzip -o {filename} > /dev/null
  df = pd.read_csv('s_OSD-' + dataset + '.txt', sep='\t', header=0)
  #puts s_OSD_XXX from zip file into dataframe
  return df

In [ ]:
def read_rnaseq_data(data):
  # data would be 'XXX_rna_seq_Normalized_Counts', or what we wanna see.
  #the OSDR "GeneLab Processed RNA..." >>"Aligned Seqence Data">> "Normalized counts data" and in there is csv for normalized rnaseq counts
  # we read m.read_rnaseq_data('XXX_rna_seq_Normalized_Counts')
  dataset = data.split('_')[0]
  url='https://osdr.nasa.gov/geode-py/ws/studies/OSD-' + str(dataset) + '/download?source=datamanager&file=GLDS-' + data + '.csv'
  #source'd be 'https://osdr.nasa.gov/geode-py/ws/studies/OSD-XXX/download?source=datamanager&file=GLDS-XXX_rna_seq_Normalized_Counts.csv
  df = pd.read_csv(url)
  #make df of url data
  return df

In [ ]:
def read_phenotype_data(dataset, data):
  #dataset = XXX from OSD-XXX
  #data is from tabular results files>>alsda processed>> all but the .csv
  #this is applicable for tonometry and phenotype stuff. obvs.
  # dataset = '557'
  # data = 'LSDS-1_immunostaining_microscopy_PNAtr_Transformed_Reusable_Results'
  url='https://osdr.nasa.gov//geode-py/ws/studies/OSD-' + str(dataset) + '/download?source=datamanager&file=' + data + '.csv'
  df = pd.read_csv(url)
  return df

# define data filtering methods

# data transformation methods

In [ ]:
def transpose_df(df, cur_index_col, new_index_col):
  #transpose rows w columns
  df = df.set_index(cur_index_col).T
  df.reset_index(level=0, inplace=True)
  cols = [new_index_col] + list(df.columns)[1:]
  df.columns = cols
  return df

# plotting methods

In [ ]:
#MODIFIED

def plotbox_summary(source,sample_key,field,treatment,space,exclude_samples=[]):
  print("Observed field: ",field, "Excluding Samples: ",exclude_samples)
  value_dict=dict()
  results=dict()
  flight=str(field)+'_flight' #category title
  nonflight=str(field+'_nonflight')
  value_dict[flight]=list()
  value_dict[nonflight]=list()    #create lists to hold flight/nonflight cols data
  for i in range(len(source)):
    if source.iloc[i][sample_key] in exclude_samples:
      continue
    elif treatment is None:
      if source.iloc[i][sample_key].startswith('F'): #F for Flight
        value_dict[flight].append(source.iloc[i][field])
      else:
        value_dict[nonflight].append(source.iloc[i][field])
    else:
      if source.iloc[i][treatment] == space:
        value_dict[flight].append(source.iloc[i][field])
      else:
        value_dict[nonflight].append(source.iloc[i][field])
  d=list(zip_longest(value_dict[flight],value_dict[nonflight],fillvalue=None))

  #statistical analysis & p-vals
  print("n flight = ", len(value_dict[flight]))
  print("n nonflight = ", len(value_dict[nonflight]))
  if len(value_dict[flight])!=0 and len(value_dict[nonflight])!=0:
    results["P-Value"]=float('%.5f' % stats.ttest_ind(value_dict[flight],value_dict[nonflight],equal_var=False).pvalue)
  print("P-Value: " , results["P-Value"])
  return d


# machine learning methods


# Differential gene expression analysis methods


In [ ]:
#ADDED IN.
def samples_to_conditions(dfT,metadata,metadata_condition_param,condition_0,condition_1,alt_cond=None,alt_cond2=None):
  #map conditions for sample comparison in deseq2
  condition_dict=dict()
  for sample in list(dfT['sample']):
    val = metadata[metadata['Sample Name']==sample][metadata_condition_param].values[0]
    if val == condition_0 or val==alt_cond or val==alt_cond2:
      condition_dict[sample]=0
    elif val == condition_1:
      condition_dict[sample]=1
    else:   #this will include anything thats not GC or Spaceflight, or alternate condition (basal..bcwe mapping it to ground)
      condition_dict[sample]=-1 #if this were 0...map Other (basal) as ground sample.


  dfT["condition"]=dfT["sample"].map(condition_dict)
  conditions_all=dfT[['sample','condition']]    #get df of all mapped conditions
  other_index=conditions_all[conditions_all['condition']==-1].index         #anything that's not Gc/Spaceflight
  conditions=conditions_all.drop(other_index)    #drop noncondition rows
  print(metadata['Sample Name'].unique()) #verify all samples included
  return conditions_all   #return conditions if ONLY wanting to deal w ground control + spaceflight exclluding basal



In [ ]:
def map_samples_to_conditions(dfT, metadata, metadata_condition_param, condition_0, condition_1):
  condition_dict=dict()
  for sample in list(dfT['sample']):
    val=metadata[metadata['Sample Name']==sample][metadata_condition_param].values[0]

    if val == condition_0:
      condition_dict[sample] = 0
    else:
      condition_dict[sample] = 1


  dfT["condition"] = dfT["sample"].map(condition_dict)
  conditions=dfT[['sample', 'condition']]

  return conditions

In [ ]:
def run_deseq2(df, metadata):
  # transpose df
  dfT = df.T
  dfT.columns=dfT.iloc[0]
  dfT=dfT.iloc[1:]
  dfT.columns.name=None
  dfT = dfT.reset_index().rename(columns={"index":"sample"})

  # map conditions
  conditions = map_samples_to_conditions(dfT, metadata, 'Factor Value[Spaceflight]', 'Ground Control', 'Space Flight')

  # get count data set up for DESeq2
  counts=dfT.drop(columns=['sample', 'condition']).reset_index(drop=True)
  counts.applymap(np.isreal)
  counts=counts.astype(int)

  # run DESeq2
  dds=DeseqDataSet(counts=counts, metadata=conditions, design_factors="condition")
  dds.deseq2()

  return dds

In [ ]:
def get_results(dds):
  # do DGEA
  stats_results=DeseqStats(dds, contrast = ('condition', 0, 1))

  # run summary
  stats_results.summary()

  # get differentially expressed genes
  res = stats_results.results_df

  return res

In [ ]:
def get_sig_genes(res, pval=0.05, l2fc=0):
  sigs = res[(res.padj < pval) & (abs(res.log2FoldChange) > l2fc)]
  return sigs

In [ ]:
def get_dge_ranked_genes(res):
  # rank genes from most to least significantly differentially expressed
  ranking = res[['stat']].dropna().sort_values('stat', ascending=False)
  ranking_index=list(ranking.index)
  ranking_index_upper=[x.upper() for x in ranking_index]
  ranking.index=ranking_index_upper

  return ranking

In [ ]:
def filter_by_dgea(data, metadata,  pval, l2fc):
  # run DESeq2
  dds = run_deseq2(data, metadata)

  # get results
  res = get_results(dds)

  # get sig genes
  sig_genes_df = get_sig_genes(res, pval=pval, l2fc=l2fc)

  # get top sig genes
  top_genes = list(sig_genes_df.sort_values('padj').index)

  # filter data by topn_genes
  return data[data['Unnamed: 0'].isin(top_genes)]


In [ ]:
#MODIFIED.
def dgea_table(data, metadata,  pval, l2fc):
  # run DESeq2
  dds = run_deseq2(data, metadata)

  # get results
  res = get_results(dds)

  return res

In [ ]:
#make links in df clicakble
def clickable_format(val):
  return f'<a target="blank"href="{val}">{val}</a>'
